<a href="https://colab.research.google.com/github/m-mehabadi/grad-maker/blob/main/_notebooks/Testing_GradientMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://www.cs.cmu.edu/~pradeepr/convexopt/Lecture_Slides/dual-ascent.pdf

In [7]:
import numpy as np
# import torch
import matplotlib.pyplot as plt

In [8]:
# dim=2000000
# a = 100*np.random.randn(4, dim) + 1123
# g = np.random.randn(dim, 1)
# b = (a-np.mean(a, axis=1)[:, np.newaxis])/np.std(a, axis=1)[:, np.newaxis]
# np.mean(b@g/dim)

In [60]:
def gradient_maker(domain_grads, epsilon=0.5, alpha=0.01, eff=0.1):
    def log():
        print(f"Iter={iter}, Condition={np.min(dgr@g/dim)}")

    dgr = domain_grads
    number_of_domains, dim = dgr.shape

    #
    _mean = np.mean(dgr, axis=1)[:, np.newaxis]
    _std = np.std(dgr, axis=1)[:, np.newaxis]
    dgr = (dgr-_mean)/_std
    
    #
    g = np.random.randn(dim)
    u_ = np.zeros(number_of_domains)
    
    iter = 0
    # log()
    while not np.min(dgr@g/dim) >= epsilon:
        if iter >= 100:
            break

        u_ = u_ + alpha*((1.+eff)*epsilon - (dgr@g)/dim)
        g = (1./number_of_domains)*np.sum(((1+u_).reshape(number_of_domains, 1))*dgr, axis=0)

        iter += 1
        # log()
    return g

### Testcases

Now let's write some test cases to make sure everything is working correctly


In [122]:
epsilon=10
alpha=0.9

In [124]:
grads = 0.000001*np.random.randn(2, 3) - 1000000
print(grads)

[[-1000000.00000128 -1000000.00000134  -999999.99999985]
 [ -999999.99999864  -999999.99999827 -1000000.00000053]]


In [125]:
g = gradient_maker(grads, epsilon=epsilon, alpha=alpha)
print(g)

[-64.54008824  57.68747067   6.99712919]


In [126]:
mean_ = np.mean(grads, axis=1)[:, np.newaxis]
std_ = np.std(grads, axis=1)[:, np.newaxis]
eps = epsilon
d = grads.shape[1]
g_bar = np.mean(g)

In [127]:
grads@g>=0

array([False, False])

In [128]:
(grads@g)[:, np.newaxis]>=eps*d*std_+d*g_bar*mean_

array([[False],
       [False]])

In [129]:
eps*d*std_+d*g_bar*mean_>=eps*d

array([[False],
       [False]])